In [1]:
%run configuration.ipynb

In [2]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model

In [4]:
class churn_model:
    
    def __init__(
        self,
        emb_dim = 128,
        dropout_rate = 0.1,
        units = 256,
        ):
        self.emb_dim = emb_dim
        self.dropout_rate = dropout_rate
        self.units = units
        self.model = None
    
    def build_model(
        self,
        ):
        
        ###
        
        input_layers = {}

        for c in feature_attributes:
            if c in even_attributes:
                input_layer = layers.Input(shape = (event_seq_len_max,), name = c)
            else:
                input_layer = layers.Input(shape = (1,), name = c)
            input_layers[c] = input_layer
        
        ##
            
        embeddding_layers = {}

        for c in attribute_vacabulary_size:
            emb_layer = layers.Embedding(
                input_dim = attribute_vacabulary_size[c],
                output_dim = self.emb_dim,
                )(input_layers[c])
            embeddding_layers[c] = emb_layer
            
        ## even sequence laeyrs
            
        concate_list = [
            embeddding_layers[c] if c in embeddding_layers
            else layers.Reshape(target_shape = (event_seq_len_max,1))(input_layers[c])
            for c in even_attributes
        ]

        event_seqence_layer = layers.Concatenate()(concate_list)
        event_seqence_layer = layers.BatchNormalization()(event_seqence_layer)
        event_seqence_layer = layers.Dropout(self.dropout_rate)(event_seqence_layer)

        event_seqence_layer = layers.Conv1D(
            filters = self.units,
            kernel_size = 1,
            strides=1,
            )(event_seqence_layer)
        event_seqence_layer = layers.Dropout(self.dropout_rate)(event_seqence_layer)
        event_seqence_layer = layers.GlobalAveragePooling1D()(event_seqence_layer)
        
        ## profile layers
        profile_feature_layers = [
            layers.Reshape(target_shape = (self.emb_dim,))(embeddding_layers[c])
            for c in feature_attributes
            if c in profile_attributes
        ]
        
        ## output layers
        feature_layer = layers.Concatenate()(
            profile_feature_layers+[event_seqence_layer]
            )

        feature_layer = layers.Dropout(self.dropout_rate)(feature_layer)
        feature_layer = layers.Dense(
            units = self.units,
            activation = 'relu',
            )(feature_layer)
        feature_layer = layers.Dropout(self.dropout_rate)(feature_layer)
        feature_layer = layers.Dense(
            units = self.units,
            activation = 'relu',
            )(feature_layer)
        feature_layer = layers.Dropout(self.dropout_rate)(feature_layer)
        output_layer = layers.Dense(
            units = 2,
            activation = 'softmax',
            )(feature_layer)
        
        ## model
        self.model = Model(
            inputs = [input_layers[c] for c in input_layers],
            outputs = output_layer
            )
        
        self.model.compile(
            optimizer="adadelta",
            loss= tf.keras.losses.CategoricalCrossentropy(),
            metrics=[
                tf.keras.metrics.AUC(),
                tf.keras.metrics.CategoricalAccuracy(name = 'acc'),
                ],
            )

model = churn_model()
model.build_model()

model.model.summary()

input_layers = {}

for c in feature_attributes:
    if c in even_attributes:
        input_layer = layers.Input(shape = (event_seq_len_max,), name = c)
    else:
        input_layer = layers.Input(shape = (1,), name = c)
    input_layers[c] = input_layer

embeddding_layers = {}

for c in attribute_vacabulary_size:
    emb_layer = layers.Embedding(
        input_dim = attribute_vacabulary_size[c],
        output_dim = emb_dim,
        )(input_layers[c])
    embeddding_layers[c] = emb_layer

# event sequence feature layer

concate_list = [
    embeddding_layers[c] if c in embeddding_layers
    else layers.Reshape(target_shape = (event_seq_len_max,1))(input_layers[c])
    for c in even_attributes
]

event_seqence_layer = layers.Concatenate()(concate_list)
event_seqence_layer = layers.BatchNormalization()(event_seqence_layer)
event_seqence_layer = layers.Dropout(dropout_rate)(event_seqence_layer)

event_seqence_layer = layers.Conv1D(
    filters = 512,
    kernel_size = 1,
    strides=1,
    )(event_seqence_layer)
event_seqence_layer = layers.Dropout(dropout_rate)(event_seqence_layer)
event_seqence_layer = layers.GlobalAveragePooling1D()(event_seqence_layer)

event_seqence_layer

# profile layers

profile_feature_layers = [
    layers.Reshape(target_shape = (emb_dim,))(embeddding_layers[c])
    for c in feature_attributes
    if c in profile_attributes
]

# output laeyr

feature_layer = layers.Concatenate()(
    profile_feature_layers+[event_seqence_layer]
    )

feature_layer = layers.Dropout(dropout_rate)(feature_layer)
feature_layer = layers.Dense(
    units = 512,
    activation = 'relu',
    )(feature_layer)
feature_layer = layers.Dropout(dropout_rate)(feature_layer)
feature_layer = layers.Dense(
    units = 512,
    activation = 'relu',
    )(feature_layer)
feature_layer = layers.Dropout(dropout_rate)(feature_layer)
output_layer = layers.Dense(
    units = 2,
    activation = 'softmax',
    )(feature_layer)

# model

model = Model(
    inputs = [input_layers[c] for c in input_layers],
    outputs = output_layer
    )

model.compile(
    optimizer="adadelta",
    loss='bianry_crossentrory',
    metrics=[
        tf.keras.metrics.AUC(),
        tf.keras.metrics.Accuracy(),
        ],
    )

# end